# Day 7 - Lab 1: Advanced Agent Workflows with MCP (Solution)

**Objective:** Learn to structure complex prompts for AI agents using the Model Context Protocol (MCP) to improve reliability and clarity, and build a code refactoring agent that leverages this protocol.

**Introduction:**
This solution notebook provides the complete code and explanations for using the Model Context Protocol. It covers manual formatting, programmatic construction with the SDK, and seamless integration with LangChain agents via adapters.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('model_context_protocol')
install_if_missing('langchain_mcp_adapters')

from utils import setup_llm_client, get_completion
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Manually Formatting an MCP Prompt

**Explanation:**
This first step demonstrates the core idea of MCP. We create a standard string, but we embed XML-like tags to give the LLM structural cues. The `<request>` tag contains the user's direct command, while the `<context>` tag groups together all the supporting information. Inside the context, `<code>` and `<instructions>` provide specific, distinct pieces of information for the agent to use. This separation helps the model differentiate between the task and the data it needs to perform the task.

In [ ]:
code_to_refactor = "def my_func(a,b):\n  x=a+b\n  y=x*2\n  return y"

manual_mcp_prompt = f"""
<request>Please refactor this code.</request>
<context>
  <code>
    {code_to_refactor}
  </code>
  <instructions>
    Make this function more readable and add Python type hints.
  </instructions>
</context>
"""

print("--- Sending Manually Formatted MCP Prompt ---")
response = get_completion(manual_mcp_prompt, client, model_name, api_provider)
print(response)

### Challenge 2 (Intermediate): Using the MCP SDK to Build a Prompt

**Explanation:**
Programmatically building the request with the MCP SDK is far superior to manual string formatting because it eliminates the risk of syntax errors and makes the prompt construction more modular, readable, and maintainable.

1.  We create `Code` and `Instructions` objects, which are specific types of context items.
2.  We group them into a `Context` object.
3.  We create the top-level `Request` object.
4.  The `.render()` method handles the conversion of these Python objects into the correctly formatted XML string, ensuring syntactical correctness.

In [ ]:
from model_context_protocol import Request, Context, Code, Instructions

# 1. Create Code and Instructions items
code_item = Code(content=code_to_refactor, language="python")
instructions_item = Instructions(content="Make this function more readable and add Python type hints.")

# 2. Create the Context object
context_obj = Context(items=[code_item, instructions_item])

# 3. Create the final Request object
mcp_request = Request(
    content="Please refactor this code.",
    context=context_obj
)

# 4. Render the request to a string
rendered_prompt = mcp_request.render()

print("--- Programmatically Rendered MCP Prompt ---")
print(rendered_prompt)

### Challenge 3 (Advanced): Building a Context-Aware Refactoring Agent

**Explanation:**
This pattern of using context builders and an MCP template is incredibly powerful for building production-ready agents. It allows you to create a stable, predictable interface for your agent, where you can easily add or remove different types of context without having to rewrite messy prompt strings.

1.  **Context Builders:** We define which types of context our agent can accept using `CodeContextBuilder` and `InstructionsContextBuilder`. These builders know how to find the relevant data from the input dictionary.
2.  **`McpChatPromptTemplate`:** This special prompt template is the core of the adapter. It takes our context builders and a request template. When the chain is invoked, it automatically uses the builders to find and format the context, then injects it into the final prompt sent to the LLM.
3.  **Invocation:** The input to the `.invoke()` call is a simple dictionary. The keys (`code`, `instructions`, `user_request`) match the context builders and the template variables, allowing the adapter to correctly assemble the final, complex MCP prompt behind the scenes.

In [ ]:
from langchain_mcp_adapters import McpChatPromptTemplate
from langchain_mcp_adapters.context_builders import CodeContextBuilder, InstructionsContextBuilder
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model_name)

# 1. Create a list of context builders
context_builders = [
    CodeContextBuilder(variable_name="code", language="python"),
    InstructionsContextBuilder(variable_name="instructions"),
]

# 2. Create the McpChatPromptTemplate
mcp_prompt_template = McpChatPromptTemplate(
    context_builders=context_builders,
    request_template="{user_request}",
)

# 3. Create the LangChain agent chain
refactoring_agent = mcp_prompt_template | llm | StrOutputParser()

# 4. Invoke the agent with a dictionary of inputs
refactoring_input = {
    "user_request": "Please refactor this code to be more Pythonic.",
    "code": "def f(data):\n  r = []\n  for i in data:\n    if i % 2 == 0:\n      r.append(i*i)\n  return r",
    "instructions": "Use a list comprehension and add type hints."
}

print("--- Invoking MCP-powered Refactoring Agent ---")
refactored_code = refactoring_agent.invoke(refactoring_input)
print(refactored_code)

## Lab Conclusion

Excellent work! You have learned how to use the Model Context Protocol to create structured, reliable prompts for your AI agents. You progressed from manually writing MCP-formatted strings to using the SDK for programmatic construction, and finally to using the LangChain adapter for seamless integration. This skill is crucial for building advanced agents that need to handle diverse and complex contextual information in a predictable way.

> **Key Takeaway:** Structuring prompts with a clear protocol like MCP makes agents more reliable. It separates the user's *request* from the *context* the agent needs, reducing ambiguity and allowing you to build more predictable, production-ready AI systems.